# **LAB ONE: INTRO TO NMR**

This lab briefly introduces nuclear magnetic resonance and covers basic NMR concepts: $\vec B_0$ and $\vec B_1$, Larmor frequency, probe tuning, free induction decay, flip angle, pulse calibration, $\vec B_0$ inhomogeneity, shimming, and spin echoes.

 -------------------------------------------------------------------------------------------------------------------------------------------------------
 #### **Required Resources**
 
 For this lab you will need:
 - ilumr fitted with 10mm RF Probe
 - Shim Sample (#001)
 
 -------------------------------------------------------------------------------------------------------------------------------------------------------

> -------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Setup Task: Run the Notebook**
> 
> 1. Edit the cell below to set the `LAB_USER_NAME` variable to your name
> 2. Click **Run->Run All Cells** in the in top menu bar of jupyterlab
> 3. Open the Table of Contents side-bar on the left edge of jupyterlab to aid in navigation
> 
> -------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
LAB_USER_NAME = 'REPLACE_ME'

In [ ]:
import panel as pn
import os
import yaml
import numpy as np

from matipo import Sequence, SEQUENCE_DIR, GLOBALS_DIR, DATA_DIR, Unit

from matipo.experiment.base_experiment import BaseExperiment, auto_inputs, PlotInterface
from matipo.experiment.plots import SignalPlot, SpectrumPlot, WobblePlot, LinePlot

pn.extension(inline=True)

WORKSPACE = os.path.join(LAB_USER_NAME, 'Lab 1')
LAB_DIR = os.path.join(DATA_DIR, WORKSPACE)
os.makedirs(LAB_DIR, exist_ok=True)
SHIM_FILE = os.path.join(LAB_DIR, 'shims.yaml')
DEFAULT_SHIMS = dict(shim_x=0, shim_y=0, shim_z=0, shim_z2=0, shim_zx=0, shim_zy=0, shim_xy=0, shim_x2y2=0)
if not os.path.isfile(SHIM_FILE):
    with open(SHIM_FILE, 'w') as f:
        yaml.dump(DEFAULT_SHIMS, f)
print('Data will be saved to', LAB_DIR)

----------------------------------------------------------------------

## 1. Introduction

### Theory

<figure style="float: right;">
<img src="Images/PrecessionDiagram.png" width="200">
<figcaption style="width: 200px;">Figure 1.1: Hydrogen nucleus precessing in a magnetic field</figcaption>
</figure>

Nuclear Magnetic Resonance occurs when a magnetic nucleus (e.g. hydrogen) is placed in an external magnetic field. Using a classical physics model, the nucleus can be thought of as a tiny bar magnet with an orientation and strength represented by its magnetic moment, written as $\vec\mu$. Nuclei with nonzero magnetic moment also have angular momentum, so it can also be thought of as a spinning top with the magnetic axis oriented in the same direction as the rotation axis (see the diagram on the right). When placed in an external magnetic field, which we will call $\vec B_0$, the nucleus will experience a torque pulling it into alignment with $\vec B_0$. However, because the nucleus also has angular momentum it will not rotate directly into alignment, but instead it will precess around $\vec B_0$; behaving like a spinning top that is gradually falling over.

The frequency of this precession is called the Larmor Frequency, given by:

$$f = \frac{\gamma}{2\pi} B_0 \tag{1.1}$$

$\gamma$ is the gyromagnetic ratio, which is a property of the nucleus. We will be using hydrogen nuclei (the hydrogen in water), which has a gyromagnetic ratio of:

$$\gamma/2\pi = 42.58 \times 10^{6} \ \mathrm{Hz}/\mathrm{T} \tag{1.2}$$

### Hardware

<figure style="float: right;">
<img src="Images/MagnetCore_ilumr.png" width="400" style="display: block;"/>
<center><figcaption style="width: 300px;">Figure 1.2: Magnetic Field Directions </figcaption></center>
<img src="Images/MagneticFieldDirection.png" width="400" style="display: block;"/>
<center><figcaption style="width: 300px;">Figure 1.3: Conventional Orientation of Fields </figcaption></center>
</figure>

In ilumr, the main magnetic field, $\vec B_0$, is created by a cylindrical permanent magnet around the sample as shown in Figure 1.2, with a direction across the bore. When a sample is inserted into the bore, the nuclei will begin to precess around $\vec B_0$. The individual magnetic moments, $\vec \mu$, of all the nuclei in the sample can be combined into a net magnetic moment, $\vec M$.

If a coil is placed around the sample, changes in the component of $\vec M$ along the coil's axis will produce a measurable current in the coil. The plane that the nuclei precess in is perpendicular to the main magnetic field, $\vec B_0$, so the coil should also be perpendicular to $\vec B_0$ to maximise the signal. Conventionally, the coil axis is labeled the $x$ axis and the magnetic field direction the $z$ axis. When the nuclei are precessing, $M_x$ will be oscillating sinusoidally due to the rotation of $\vec M$ around the $z$ axis. The orientations of $\vec B_0$, $\vec B_1$, and $\vec M$ are shown in Figure 1.3 in the conventional coordinate system.

After waiting some time, the nuclei in the sample will have reached an equilibrium where $\vec M$ is completely aligned with $\vec B_0$ and will not induce any signal in the coil. To obtain a signal, the coil is driven to create a magnetic field $\vec B_1$ that oscillates at the Larmor frequency, exciting the resonance. After excitation, $\vec M$ will precess while gradually returning to the equilibrium position, creating a signal in the coil called the Free Induction Decay (FID).

## 2. Tuning the RF Probe

<figure style="float: right;">
<img src="Images/CapCircuit.png" width="300" style="display: block;">
<center><figcaption style="width: 300px;">Figure 2.1: Resonant Circuit</figcaption></center>
</figure>

In order to efficiently generate the $\vec B_1$ field and pickup the signal from the net magnetization $\vec M$, the coil is combined with a capacitor to form a resonant circuit shown in the figure on the right. The resonant frequency of an LC circuit is:

$$f = \frac{1}{2\pi\sqrt{LC}} \tag{2.1}$$

This frequency needs to be close to the NMR resonant frequency to optimise the transmit and receive efficiency of the probe. Adjusting the *Tune* capacitor on the probe changes the value of $C$. The LC resonator circuit also needs to be impedance matched to the RF transmit amplifier and receive preamplifier, which have a characteristic impedance of 50 Ohms. This is accomplished with a *Match* capacitor.

The *Match* capacitor actually also has some effect on the resonant frequency, and the *Tune* capacitor has some effect on the impedance, so in practice it is an iterative process to optimise these values.

The Wobble tool sweeps through a range of frequencies and measures the amount of power reflected by the probe circuit. The probe will only absorb power close to its resonant frequency, so reflected power plot will have a dip at that frequency. At the resonant frequency there will still be some power reflected if there is an impedance mismatch, so the level of the reflected power plot at the bottom of the dip indicates how well matched the probe is.

<figure style="float: right;">
<img src="Images/ilumr_tuning_caps.png" width="250" style="display: block;"/>
</figure>

<figure style="float: right;">
<img src="Images/ilumr_Diskanimation.gif" width="250" style="display: block;"/>
</figure>

> -------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Task 2.1: Tune the Probe**
> 1. For ilumr the magnetic field strength is approximately $B_0 = 0.35 \ \mathrm{T}$ (if a hall probe is available you may measure the field strength to get a more accurate value).
> 2. Using [Equation 1.1](#mjx-eqn-1), calculate the expected Larmor frequency and enter your calculated value in MHz into the "Frequency" box below.
> 3. Remove the black plate on top of the ilumr by rotating it anticlockwise.
> 4. Run the **Wobble** tool below in loop mode. 
> 5. While the **Wobble** tool is running, adjust the variable capacitors with the provided screwdriver to tune and match the RF coil to your calculated Larmor frequency (Note: the dotted line on the plot corresponds to the desired frequency). 
> 6. Record the the dB value of the lowest point. 
> 7. Save an image of the reflected power plot.
> 8. Load a sample into the ilumr. Note: Loading the sample into the coil will slightly affect both the tune and match of the coil.
> -------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
class WobbleExperiment(BaseExperiment):
    def setup(self):
        self.workspace = WORKSPACE
        self.seq = Sequence(SEQUENCE_DIR+'wobble.py')
        self.enable_runloop=True
        self.plots = {
            'wobble': WobblePlot(figure_opts=dict(height=200))
        }
        
        self.inputs = auto_inputs(self.seq, {
            'f': 14.0*Unit('MHz')
        })
        
        # set custom label
        self.inputs['f'].name = 'Frequency (MHz)'

    def update_par(self):
        self.seq.setpar(f_bw=1e6)
        # if the frequency range would overlap 0, shift the centre frequency to avoid negative frequencies
        if self.seq.par.f < self.seq.par.f_bw/2:
            self.seq.setpar(f=self.seq.par.f_bw/2)
    
    async def update_plots(self, *args, **kwargs):
        await self.seq.fetch_data()
        self.plots['wobble'].update(
            seqdata=self.seq.data, 
            f=self.seq.par.f, 
            f_bw=self.seq.par.f_bw, 
            n_samples=self.seq.par.n_samples)
            

exp1 = WobbleExperiment(state_id='exp1')
exp1()

NOTE: If using **Run Loop**, click abort before continuing


## 3. The Free Induction Decay Signal

<figure style="float: right;">
<img src="Images/depth.png" width="100" style="display: block;">
</figure>

> ---
> #### **Task 3.1: Pulse & Collect** 
> 1. Use the depth gauge to align the **shim sample** with the 10mm point.
> 2. Load the **shim sample** into the ilumr.
> 3. Input your calculated frequency and a starting RF amplitude guess of 10% into the boxes below.
> 4. Run the **Pulse Collect** tool below. The result should be a visible decaying sine wave in the signal plot, and a narrow spike in the spectrum plot. If there is no visible NMR signal, increase the Pulse Amplitude and try again.
> 5. Zoom into the spike in the spectrum plot and read the relative frequency off the frequency axis (use the zoom tools at the top right of the plot, and take note of the SI prefix on the axis tick labels).
> 6. Add the frequency offset you measured to the frequency input and Rerun **Pulse Collect**. The spike in the spectrum should now be centred at 0 on the relative frequency axis.
> 7. Record the frequency you have measured, and use [Equation 1.1](#mjx-eqn-1) to calculate the field strength from this measured frequency. How does the field strength found this way compare to the field strength provided in **Task 2.1**?
> ---

In [ ]:
# Pulse collect experiment
class PulseCollectExperiment(BaseExperiment):
    def setup(self):
        self.workspace = WORKSPACE
        self.seq = Sequence(SEQUENCE_DIR+'FID.py')
        self.enable_partialplot=False
        self.default_par_files = [SHIM_FILE]
        self.plots = {
            'signal': SignalPlot(show_magnitude=True),
            'spectrum': SpectrumPlot()
        }
        
        self.inputs = auto_inputs(self.seq, {
            'f': 14.0*Unit('MHz'),
            'a_90': 10*Unit('%')
        })
        
        # set custom label
        self.inputs['f'].name = 'Frequency (MHz)'
        self.inputs['a_90'].name = 'Pulse Amplitude (%)'
        
        self.fixed_par = dict(
            n_scans=4,
            t_90=32e-6,
            n_samples=1000,
            t_dw=0.2e-6,
            t_end=0.5
        )

    def update_par(self):
        self.seq.setpar(**self.fixed_par)
    
    async def update_plots(self, *args, **kwargs):
        await self.seq.fetch_data()
        self.plots['signal'].update(seqdata=self.seq.data, t_dw=self.seq.par.t_dw)
        self.plots['spectrum'].update(seqdata=self.seq.data, t_dw=self.seq.par.t_dw)
            

exp2 = PulseCollectExperiment(state_id='exp2')
exp2()

### Note: Quadrature Detection and Rotating Reference Frames

At the NMR frequency you measured, there would be thousands of cycles in the signal plot above if it were just a measurement of the magnetization along a fixed axis. Instead, to simplify processing and visualization, the signal from the probe is processed using a quadrature detector which removes a carrier frequency from a signal; this is a commonly used technique in radio receivers.

The result is a measurement of the signal in a rotating reference frame. Imagine a camera that is stationary and one that is rotating at your chosen frequency around the sample (see Figure 3.1); the stationary camera will see the magnetization precessing rapidly at the NMR frequency, however the rotating camera will only see the difference between the movement of the magnetization and the movement of the camera. The perspectives of these imaginary cameras are often called the "lab frame" and "rotating frame". If the rotation frequency of the rotating frame is perfectly set to the NMR frequency, then the measured signal does not oscillate at all!

The real and imaginary parts of the signal are the components along the $y'$ and $x'$ axes in the rotating frame, respectively.

<center><img src="Images/RotatingFrame.png" width="1400"></center>
<center><figcaption style="width: 300px;">Figure 3.1: Rotating Frames </figcaption></center>

> ---
> #### **Task 3.2: Retune the Probe**
> 1. If the frequency measured in **Task 3.1** is significantly different from that estimated in **Task 2.1** then probe may need to be retuned.
> 2. Run the **Wobble** tool below with the new frequency, and retune the probe if necessary. As a rule of thumb, the lowest point should be within 20kHz of the NMR frequency (dotted line).
> ---


In [ ]:
exp3 = WobbleExperiment(state_id='exp3')
exp3()

## 4. Optimising the Flip Angle

<figure style="float: right;  margin-top: 0;">
<center><img src="Images/SpinAngleDiagram.png" width="600" style="display: block;"></center>
<center><figcaption style="width: 300px;">Figure 4.1: Relationship between pulse amplitude, flip angle and signal</figcaption></center>
</figure>

In the rotating frame, $\vec B_1$ appears like a static field (because it is rotating in the lab frame at the same frequency as the rotating frame). In this case, $\vec B_1$ is always aligned with the $x'$ axis, and exerts a torque on the nuclei. This torque causes the nuclei to precess around the $x'$ axis (while still also precessing around $\vec B_0$). If the $\vec B_1$ field is applied for a short time $t$, the final angle of rotation will be (in radians):

$$\alpha = \gamma B_1 t \tag{4.1}$$

where $\gamma$ is the gyromagnetic ratio. 

As a result of this relationship, the flip angle $\alpha$ is proportional to the area under the RF pulse in the pulse sequence.

The signal we measure is proportional only to the component of $\vec M$ that is in the $xy$ plane, called $\vec M_{xy}$, and the component along the $z$ axis is invisible to the probe. The RF pulse amplitude guess we used in **Task 3.1** resulted in a signal but, to maximise the signal, the pulse amplitude needs to be calibrated to cause a $90^\circ$ rotation of the magnetization from the $z$ axis to the $xy$ plane. Figure 4.1 shows how a flip angle that is less or greater then $90^\circ$ results in reduced signal due to $\vec M$ not lying in the $xy$ plane.

The $B_1$ field strength depends on the RF pulse amplitude ($A_{1-3}$), but is also affected by the probe tuning and loading from the sample, so the correct RF pulse amplitude must be empirically determined each time the probe tuning or sample is changed. This will be done in **Task 4.1**.

> ---
> #### **Task 4.1: Pulse Calibration** 
> 1. Set the frequency and run the **Pulse Calibration** tool below.
> 2. Find the amplitude value that corresponds to a $90^\circ$ flip angle (the first maximimum of the *Calibration* plot) and record it in your notebook.
> ---

In [ ]:
# pulse calibration experiment

def gaussian_apodize(data, lb):
    t = np.linspace(0, 1, len(data))
    return data * np.exp(-lb*lb*t*t)

class PulseCalibrationExperiment(BaseExperiment):
    def setup(self):
        self.workspace = WORKSPACE
        self.seq = Sequence(SEQUENCE_DIR+'FID.py')
        self.enable_partialplot=True
        self.default_par_files = [SHIM_FILE]
        self.plots = {
            'signal': SignalPlot(show_magnitude=True),
            'cal': LinePlot(figure_opts=dict(
                title='Calibration',
                x_axis_label='Pulse Amplitude (%)',
                y_axis_label='Signal Sum of Squares (V²)',
                tooltips=[
                    ('x', '@x'),
                    ('y', '@y')
                ]))
        }
        
        self.inputs = auto_inputs(self.seq, {
            'f': 14.0*Unit('MHz')
        })
        
        # set custom label
        self.inputs['f'].name = 'Frequency (MHz)'
    
    async def update_plots(self, final, *args, **kwargs):
        self.plots['signal'].update(seqdata=self.y_latest, t_dw=self.seq.par.t_dw)
        self.plots['cal'].update(data=dict(sumsq=dict(x=100*self.amp_vals, y=self.sumsqs)))
    
    async def run(self, progress_handler):
        self.seq.setpar(
            t_90=32e-6,
            n_scans=1,
            n_samples=2000,
            t_dw=0.5e-6,
            t_acqdelay=50e-6,
            t_end=0.5
        )
        
        self.y_latest = None # keep copy of latest signal data for plotting
        self.N_amp = 51
        self.amp_vals = 0.02*np.arange(self.N_amp)
        self.sumsqs = np.empty(self.N_amp)
        self.sumsqs[:] = np.nan
        
        for i, amp in enumerate(self.amp_vals):
            self.seq.setpar(a_90=amp)
            self.y_latest = await self.seq.run()
            self.sumsqs[i] = np.sum(gaussian_apodize(np.abs(self.y_latest), 2) ** 2)
            progress_handler(i, len(self.amp_vals))

exp4 = PulseCalibrationExperiment(state_id='exp4')
exp4()

> ---
> #### **Task 4.2: FID Check after Pulse Calibration** 
> 1. Enter your frequency and the RF pulse amplitude value you determined in **Task 4.1** below.
> 2. Run the **Pulse Collect** experiment and compare the initial signal level to what you had in **Task 3.1**.
> 3. Save an image of the signal plot.
> 4. Measure the decay time constant of the signal:
>     1. Measure the initial magnitude of the signal.
>     2. Measure the time where the magnitude has decayed to approximately 37% ($1/e$) of the initial magnitude.
> 5. Record this time constant in your notebook.
> ---

In [ ]:
exp5 = PulseCollectExperiment(state_id='exp5')
exp5.fixed_par['t_dw'] = 5e-6
exp5()

## 5. Signal Decay & Shimming

The NMR signal is short lived, after an excition the net magnetization $\vec M$ gradually returns to the $B_0$ direction ($z$ axis). This relaxation is described by two processes:

*Longitudinal relaxation* is the recovery of the *longitudinal* component $M_z$ with time constant $T_1$. In the simple case after a $90^\circ$ pulse this is:

$$M_z(t) = M_0 \cdot (1-e^{-t/T_1}) \tag{5.1}$$

where $M_0$ is the equilibrium magnetization, obtained after waiting a long time.

*Transverse relaxation*: The decay of the *transverse* component $M_{xy}$ with time constant $T_2$:

$$M_{xy}(t) = M_{xy}(0) \cdot e^{-t/T_2} \tag{5.2}$$

<figure style="float: right;">
<img src="Images/T2StarDecay.png" width="300">
<figcaption style="width: 300px">Figure 5.1: The FID in a homogeneous field ($T_2$) vs inhomogeneous field ($T_2^*$)</figcaption>
</figure>

The transverse relaxation responsible for our signal decaying is partly caused by random processes in the sample itself that dephase the individual magnetic moments of nuclei, but is also affected by inhomogeneity (variation at different locations) in the main magnetic field $\vec B_0$ (see figures 5.1 & 5.2). The time constant due to inhomogeneity is called $T_2^*$, to distinguish it from the time constant that is a property only of the sample, called $T_2$.

Improving the homogeneity of the magnetic field will lengthen $T_2^*$, making the signal last longer and improving the total amount of signal obtained from a single excitation. In practice this is accomplished with an array of coils and drivers that precisely control the current through them, which are called *electronic shims*.

<center><figure>
<img src="Images/T2StarPhaseDiagram.png">
<figcaption>Figure 5.2: Magnetic moments at different points in the sample (1,2,3,4) dephasing due to differences in local $B_0$ field strength, reducing the measureable net magnetic moment $M_{xy}$</figcaption>
</figure></center>

> ---
> #### **Task 5.1: Run Autoshim**
> 1. Update the input parameters and Run the **Autoshim** tool below. This tool starts with all shims set to zero, and is equivalent to the **Coarse** setting in the Autoshim dashboard app. This will take about 5 mins.
> - The shim values will be saved to the `shims.yaml` file in your `LAB_DIR` (printed at the top of the notebook after entering your username), and will be used by the other experiments in this notebook automatically.
> ---

In [ ]:
from matipo.util.optimisation import nelder_mead_async

class AutoshimExperiment(BaseExperiment):
    def setup(self):
        self.workspace = WORKSPACE
        self.shim_file = SHIM_FILE
        self.seq = Sequence(SEQUENCE_DIR+'FID.py')
        self.enable_partialplot=True
        self.plots = {
            'signal': SignalPlot(show_magnitude=True),
            'shims': LinePlot(
                legend_opts=dict(
                    location='bottom_left',
                    orientation='horizontal'
                ),
                figure_opts=dict(
                    title='Shim Values',
                    x_axis_label='Iteration',
                    y_axis_label='Value (%)'
                )
            )
        }
        
        self.inputs = auto_inputs(self.seq, {
            'f': 14.0*Unit('MHz'),
            'a_90': 10*Unit('%')
        })
        
        # set custom label
        self.inputs['f'].name = 'Frequency (MHz)'
        self.inputs['a_90'].name = 'Pulse Amplitude (%)'
        
        self.N_O1 = 3
        self.N_O2 = 5
        self.O1_shim_step = 0.5
        self.O2_shim_step = 0.5
        self.O1_precision = 0.005
        self.O2_precision = 0.005
        self.max_iterations = 100

        self.shim_order = ['shim_x', 'shim_y', 'shim_z', 'shim_z2', 'shim_zx', 'shim_zy', 'shim_xy', 'shim_x2y2']
    
    def saveshims(self, shims):
        with open(self.shim_file, 'w') as f:
            yaml.dump(shims, f)
    
    async def update_plots(self, final, *args, **kwargs):
        self.plots['signal'].update(seqdata=self.y_latest, t_dw=self.seq.par.t_dw)
        shim_vals_x_axis = np.arange(len(self.shim_vals['shim_x']))
        self.plots['shims'].update(data={key: dict(x=shim_vals_x_axis, y=self.shim_vals[key]) for key in self.shim_order})
    
    async def run(self, progress_handler):
        self.seq.setpar(
            t_90=32e-6,
            n_scans=1,
            n_samples=1000,
            t_dw=32e-6,
            t_acqdelay=50e-6,
            t_end=0.5
        )
        
        self.y_latest = None # keep copy of latest signal data for plotting
        self.shim_vals = {key: [] for key in self.shim_order}
        self.sumsqs = []
        
        init_step = [self.O1_shim_step]*self.N_O1 + [self.O2_shim_step]*self.N_O2
        precision = [self.O1_precision]*self.N_O1 + [self.O2_precision]*self.N_O2
        lower_bounds = [-1]*(self.N_O1+self.N_O2)
        upper_bounds = [1]*(self.N_O1+self.N_O2)
        shims = np.zeros(self.N_O1+self.N_O2)
        
        # do a dummy so that the first actual run has similar T1 relaxation effects
        await self.evalfunc(shims, dummy=True)
        
        i = 0
        best_shims = shims
        async for r in nelder_mead_async(self.evalfunc, shims, x_lb=lower_bounds, x_ub=upper_bounds, max_iter=self.max_iterations, step=init_step, x_precision=precision):
            if not np.array_equal(r[0], best_shims):
                best_shims = r[0].tolist()
                self.saveshims({key: best_shims[i] for i, key in enumerate(self.shim_order)})
            progress_handler(i, self.max_iterations)
            i += 1
        
        
    
    async def evalfunc(self, try_shims, dummy=False):
        for i, shim_name in enumerate(self.shim_order):
            self.seq.setpar(shim_name, try_shims[i])
        
        self.y_latest = await self.seq.run()
        
        # return early if it's a dummy run
        if dummy:
            return 0
        
        sumsq = np.sum(gaussian_apodize(np.abs(self.y_latest), 2) ** 2)
        self.sumsqs.append(sumsq)
        for i, shim_name in enumerate(self.shim_order):
            self.shim_vals[shim_name].append(100*try_shims[i])
        
        return -sumsq

exp6 = AutoshimExperiment(state_id='exp6')
exp6()

**Autoshim** iterates through various combinations of currents through the shims, and uses an optimisation algorithm to maximise the total integrated signal (area under the FID magnitude curve). The FID signal should lengthen as the "Shim Values" (proportional to the currents through the coils) converge to the optimal values.

> ---
> #### **Task 5.2: FID Check with Shim**
> 1. Enter the correct parameters and Run the **Pulse Collect** experiment below. Check how the amplitude and decay of the signal has changed compared to without shims in **Task 4.2**. With a good shim there should still be significant signal at 4 ms.
> 2. Save an image of the signal plot.
> 3. Measure the decay time constant in the same way as in **Task 4.2**: by finding the time when the magnitude has decayed to 37% of its initial value. This value is $T_2^*$.
> 4. Compare the $T_2^*$ measured in **Task 4.2** before shimming to the $T_2^*$ measured here after shimming. By what factor has it improved?
> ---

In [ ]:
exp7 = PulseCollectExperiment(state_id='exp7')
exp7.fixed_par['t_dw'] = 5e-6
exp7()

## 6. Spin Echo 

Shimming the field will improve the duration of the NMR signal, but the signal can also be brought back after dephasing due to inhomogeneity using the *spin echo* technique.

<figure style="float: right;">
<img src="Images/SpinEchoPulse.png" width="400">
<center><figcaption style="width: 400px">Figure 6.1: Spin Echo Pulse Sequence</figcaption></center>
</figure>

A spin echo can be formed by applying a $90^\circ$ flip angle RF pulse and a $180^\circ$ flip angle pulse in succession as shown in Figure 6.1 on the right.

Imagine two small groups of nuclei at different locations in the magnetic field which experience slightly different field strengths and therefore precession frequencies. We will call these *isochromats*, the *iso* prefix means the nuclei within an *isochromat* all have the same precession frequency. If one isochromat with magnetic moment $M_1$ has a slightly higher precession frequency and the other, $M_2$, slightly lower, then they will begin to dephase after the initial pulse as shown in Fig. 10 (a) below.

When the $180^\circ$ flip angle pulse is applied with the $\vec B_1$ direction along the $y'$ axis, the isochromat magnetic moments $\vec M_1$ and $\vec M_2$ will individually be rotated by $180^\circ$ around the $y'$ axis, as shown in Figure 6.2 (b). After the $180^\circ$ pulse the $\vec M_1$ isochromat will continue to precess faster and the $\vec M_1$ isochromat slower, but they have been flipped and are now coming back into phase as shown in Figure 6.2 (c).

<center><img src="Images/SpinEchoPhase.png" width="1300"></center>
<center><figcaption style="width: 300px;">Figure 6.2: Effect of 180 degree pulse on phase </figcaption></center>
<br/>

All of the isochromats in a sample will come back into perfect phase simultaneously at a single point in time called $T_E$, the echo time, which is where the echo signal is at maximum (see Figure 6.1). The spin echo sequence can only refocus the dephasing caused by magnet inhomogeneity ($T_2^*$); the signal will still have decayed due to random dephasing in the sample ($T_2$), so longer echo times ($T_E$) will result in less echo signal.

> ---
> #### **Task 6.1: Spin Echo**
> 1. What should the amplitude of the second RF pulse in Figure 6.2 be to achieve a $180^\circ$ flip angle?
> 2. Enter this value into the "Second Pulse Amplitude" box below, set the frequency and first pulse amplitude parameters as well, and run the experiment.
> 3. If the second pulse amplitude was set correctly, changing it by 10% either up or down should decrease the height of the echo. Is this the case? (Try zooming in on the echo to see more easily how it changes)
> 4. Measure the maximum magnitude of the FID at the beginning of the signal plot and record it.
> 5. Increase the echo time until the maximum magnitude of the echo is roughly 37% ($1/e$) of the FID initial magnitude.
> 6. This echo time approximates the $T_2$ of the sample. Record this $T_2$ value and compare it to the $T_2^*$ value after shimming measured in **Task 5.2**. Has shimming completely removed inhomogeneities from the magnetic field?
> 7. Save an image of the signal plot.
> ---

In [ ]:
class SpinEchoExperiment(BaseExperiment):
    
    def setup(self):
        self.workspace = WORKSPACE
        self.seq = Sequence('programs/full_acq_SE.py')
        self.plots = {
            'signal': SignalPlot(show_magnitude=True)
        }
        
        self.inputs = auto_inputs(self.seq, {
            'f': 14.0*Unit('MHz'),
            'a_90': 10*Unit('%'),
            'a_180': 0*Unit('%'),
            't_echo': 50*Unit('ms')
        })
        
        self.inputs['f'].name = 'Frequency (MHz)'
        self.inputs['t_echo'].name = 'Echo Time (ms)'
        self.inputs['a_90'].name = 'First Pulse Amplitude (%)'
        self.inputs['a_180'].name = 'Second Pulse Amplitude (%)'
    
    def update_par(self):
        self.seq.setpar(
            t_90=32e-6,
            t_180=32e-6,
            n_scans=1,
            n_samples=10000,
            t_dw=20e-6, # using a long dwell time for narrow bandwith to more easily see the spectrum shape
            t_end=0.2
        )
    
    async def update_plots(self, *args, **kwargs):
        await self.seq.fetch_data()
        self.plots['signal'].update(seqdata=self.seq.data, t_dw=self.seq.par.t_dw)
            

exp8 = SpinEchoExperiment(state_id='exp8')
exp8()